In [11]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [12]:
# Load the datasets
tgca_prim_window_meth_df = pd.read_csv("../_INPUTS_/meth_cols_tgca_prim.tsv", sep='\t')
tgca_sith_corr_meth_df = pd.read_csv("../_INPUTS_/meth_cols_tgca_sith.tsv", sep='\t')
tgca_iqr_corr_meth_df = pd.read_csv("../_INPUTS_/meth_cols_tgca_iqr.tsv", sep='\t')

tgca_sith_df = pd.read_csv("../_INPUTS_/tcga_sith_data.csv")
tgca_sith_df_trim = tgca_sith_df[['bcrSID', 'sample_type', "SITH", "INT_SITH_IQR" ]]
tgca_sith_df = tgca_sith_df_trim

In [13]:
# rename Sample ID to bcrSID to match sith dataset
tgca_prim_window_meth_df.rename(columns={'Sample ID': 'bcrSID'}, inplace=True)
tgca_sith_corr_meth_df.rename(columns={'Sample ID': 'bcrSID'}, inplace=True)
tgca_iqr_corr_meth_df.rename(columns={'Sample ID': 'bcrSID'}, inplace=True)

# rename INT_SITH_IQR to INT_IQR in sith dataset
tgca_sith_df.rename(columns={'INT_SITH_IQR': 'INT_IQR'}, inplace=True)

# add sample_code col
tgca_sith_df['sample_code'] = tgca_sith_df['sample_type'].apply(lambda x: 1 if x == 'Primary Tumor' else 2)

In [14]:
tgca_sith_df

,bcrSID,sample_type,SITH,INT_IQR,sample_code
0,TCGA-04-1331-01A,Primary Tumor,0.415997,0.283701,1
1,TCGA-04-1347-01A,Primary Tumor,0.408280,0.335263,1
2,TCGA-04-1349-01A,Primary Tumor,0.481373,0.246165,1
3,TCGA-04-1367-01A,Primary Tumor,0.421612,0.260305,1
4,TCGA-04-1371-01A,Primary Tumor,0.719342,0.369128,1
...,...,...,...,...,...
744,TCGA-QG-A5YV-01A,Primary Tumor,0.322534,0.278484,1
745,TCGA-QG-A5YW-01A,Primary Tumor,0.297344,0.235468,1
746,TCGA-QG-A5YX-01A,Primary Tumor,0.426095,0.310176,1
747,TCGA-QG-A5Z1-01A,Primary Tumor,0.406377,0.316370,1


In [15]:
tgca_prim_window_meth_df

,bcrSID,cg16000360,cg27179327,cg13991953,cg08268266,cg23724117,cg01910484,cg04603812,cg25471956,cg18756868,...,cg15933935,cg10918927,cg15901722,cg06872331,cg14255426,cg03651478,cg18160135,cg19720672,cg02878216,cg09851528
0,TCGA-05-4396-01A,0.101623,0.015880,0.066446,0.160910,0.038628,0.020577,0.017683,0.059806,0.014721,...,0.030823,0.083624,0.125749,0.067617,0.108804,0.046714,0.090956,0.013975,0.034752,0.056823
1,TCGA-05-5429-01A,0.089167,0.017597,0.064777,0.084332,0.041673,0.015860,0.456887,0.066462,0.011739,...,0.025787,0.127243,0.131426,0.072001,0.321529,0.037486,0.089327,0.019974,0.043990,0.042351
2,TCGA-06-0125-01A,0.122308,0.030056,0.064253,0.098278,0.056320,0.020288,0.015539,0.080279,0.014340,...,0.033135,0.080412,0.525168,0.107706,0.474479,0.064359,0.488356,0.041173,0.068457,0.128585
3,TCGA-06-0125-02A,0.159563,0.018080,0.099393,0.083687,0.044774,0.023586,0.021588,0.076114,0.013959,...,0.032333,0.104453,0.445625,0.147981,0.547848,0.099223,0.343584,0.017427,0.056076,0.117151
4,TCGA-06-0152-01A,0.260319,0.030138,0.132030,0.103450,0.083753,0.021126,0.024468,0.093113,0.013225,...,0.030544,0.096536,0.200592,0.113904,0.116355,0.073626,0.339126,0.033900,0.058704,0.392209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,TCGA-QG-A5YV-01A,0.050493,0.017392,0.070687,0.068855,0.059284,0.026036,0.016424,0.061240,0.016659,...,0.037680,0.109253,0.079010,0.052352,0.106406,0.042101,0.049598,0.025292,0.036968,0.050845
598,TCGA-QG-A5YW-01A,0.119887,0.014824,0.092168,0.085042,0.064425,0.024068,0.018714,0.047773,0.015516,...,0.040147,0.093837,0.145196,0.048348,0.099541,0.041290,0.112768,0.019731,0.035608,0.085552
599,TCGA-QG-A5YX-01A,0.047293,0.017620,0.058332,0.076901,0.052872,0.024609,0.018738,0.052206,0.015906,...,0.034410,0.089688,0.062822,0.046186,0.095410,0.043634,0.052090,0.019923,0.035089,0.068825
600,TCGA-QG-A5Z1-01A,0.087043,0.014834,0.078576,0.076830,0.054820,0.022155,0.017476,0.067265,0.019254,...,0.047422,0.118007,0.101908,0.056815,0.277947,0.045796,0.079663,0.016444,0.052410,0.054664


In [16]:
# Merge the datasets on the first column (assumed to be the index 0)
tgca_prim_window_meth_df_merged = pd.merge(tgca_sith_df, tgca_prim_window_meth_df, on=tgca_sith_df.columns[0])
# Save the merged dataset to a new TSV file
tgca_prim_window_meth_df_merged.to_csv("../_OUTPUTS_/merged_sith_meth_tgca_prim_window.tsv", sep='\t', index=False)
# Display the first few rows of the merged dataset
print(tgca_prim_window_meth_df_merged.head())

# Merge the datasets on the first column (assumed to be the index 0)
tgca_sith_corr_meth_df_merged = pd.merge(tgca_sith_df, tgca_sith_corr_meth_df, on=tgca_sith_df.columns[0])
# Save the merged dataset to a new TSV file
tgca_sith_corr_meth_df_merged.to_csv("../_OUTPUTS_/merged_sith_meth_tgca_sith_corr.tsv", sep='\t', index=False)
# Display the first few rows of the merged dataset
print(tgca_sith_corr_meth_df_merged.head())

# Merge the datasets on the first column (assumed to be the index 0)
tgca_iqr_corr_meth_df_merged = pd.merge(tgca_sith_df, tgca_iqr_corr_meth_df, on=tgca_sith_df.columns[0])
# Save the merged dataset to a new TSV file
tgca_iqr_corr_meth_df_merged.to_csv("../_OUTPUTS_/merged_sith_meth_tgca_iqr_corr.tsv", sep='\t', index=False)
# Display the first few rows of the merged dataset
print(tgca_iqr_corr_meth_df_merged.head())


             bcrSID      sample_type      SITH   INT_IQR  sample_code  \
0  TCGA-05-4396-01A    Primary Tumor  0.364670  0.327785            1   
1  TCGA-05-5429-01A    Primary Tumor  0.672152  0.202033            1   
2  TCGA-06-0125-02A  Recurrent Tumor  0.442042  0.276554            2   
3  TCGA-06-0152-01A    Primary Tumor  0.403920  0.302980            1   
4  TCGA-06-0171-01A    Primary Tumor  0.514956  0.245369            1   

   cg16000360  cg27179327  cg13991953  cg08268266  cg23724117  ...  \
0    0.101623    0.015880    0.066446    0.160910    0.038628  ...   
1    0.089167    0.017597    0.064777    0.084332    0.041673  ...   
2    0.159563    0.018080    0.099393    0.083687    0.044774  ...   
3    0.260319    0.030138    0.132030    0.103450    0.083753  ...   
4    0.172404    0.035175    0.117970    0.105973    0.046501  ...   

   cg15933935  cg10918927  cg15901722  cg06872331  cg14255426  cg03651478  \
0    0.030823    0.083624    0.125749    0.067617    0.108804  

In [17]:
# Generate summary statistics
summary_statistics = tgca_prim_window_meth_df_merged.describe()

# Display the summary statistics
print(summary_statistics)

             SITH     INT_IQR  sample_code  cg16000360  cg27179327  \
count  509.000000  507.000000   509.000000  509.000000  509.000000   
mean     0.490801    0.271539     1.058939    0.140583    0.019083   
std      0.132050    0.056230     0.235742    0.107315    0.010423   
min      0.128773    0.000000     1.000000    0.035219    0.009978   
25%      0.398242    0.247345     1.000000    0.079956    0.014824   
50%      0.490857    0.277963     1.000000    0.108941    0.017320   
75%      0.584324    0.304188     1.000000    0.157895    0.020754   
max      0.920000    0.409503     2.000000    0.804082    0.195603   

       cg13991953  cg08268266  cg23724117  cg01910484  cg04603812  ...  \
count  509.000000  472.000000  488.000000  508.000000  509.000000  ...   
mean     0.086949    0.099360    0.063865    0.034649    0.051004  ...   
std      0.037369    0.066998    0.050213    0.074516    0.139800  ...   
min      0.046381    0.031124    0.027892    0.012626    0.007214  ...   